In [1]:

# coding: utf-8

# In[ ]:


from __future__ import print_function

import sys, os
import numpy as np
import struct
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
os.environ['XILINX_XRT'] = '/usr'
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")
    print("===============================")
    
    # Overlay
    ol = Overlay("FFT_NTT.bit")
    ipFN = ol.FFT_NTT_0

    # Memory allocate
    Buffer0 = allocate(shape=(1024,), dtype=np.float64)
    Buffer1 = allocate(shape=(1024,), dtype=np.uint64)
    fft_out = [0 for i in range(1024)]
    ntt_out = [0 for i in range(1024)]

    # =============== NTT test vector =================
    with open("mq_ntt_in.txt", "r") as NTT_in:
        for i in range(1024):
            Buffer1[i] = np.uint16(NTT_in.readline())
    print(bin(Buffer1[0]))
    
    timeKernelStart = time()

    mode = 0 # NTT_req
    ipFN.write(0x1C, 0) # Coefficient
    ipFN.write(0x10, Buffer1.device_address)
    # ================= Start NTT =====================
    ipFN.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
    while (ipFN.read(0x00) & 0x4) == 0x0:
        continue
    timeKernelEnd = time()
    print("Kernel execution time [mode: NTT]: " + str(timeKernelEnd - timeKernelStart) + " s")

    for i in range(1024):
        ntt_out[i] = Buffer1[i]
    
    # =============== FFT test vector =================
    with open("FFT_in.txt", "r") as FFT_in:
        for i in range(1024):
            Buffer0[i] = float(FFT_in.readline())

    timeKernelStart = time()

    mode = 1 # FFT_req
    ipFN.write(0x1C, 1) # Coefficient
    ipFN.write(0x10, Buffer0.device_address)
    # ================= Start FFT =====================
    ipFN.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
    while (ipFN.read(0x00) & 0x4) == 0x0:
        continue
    timeKernelEnd = time()
    print("Kernel execution time [mode: FFT]: " + str(timeKernelEnd - timeKernelStart) + " s")

    for i in range(1024):
        fft_out[i] = Buffer0[i]

    
    # ================== Compare ======================
    print("-------------------------------")
    print("Compare FFT with golden vector:")
    error_cnt = 0
    round_factor = 6
    with open("FFT_out.txt", "r") as FFT_out:
        for i in range(1024):
            golden = float(FFT_out.readline())
            if round(fft_out[i], round_factor) != round(golden, round_factor):
                print(f"{i:<4}Failed!! Expect value: {golden:.17f}. Actual value: {fft_out[i]:.17f}")
                error_cnt += 1
            #else:
                #print(f"Calculate value: {outBuffer0[i]:.17f}")
    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")
    
    print("-------------------------------")
    print("Compare NTT with golden vector:")
    error_cnt = 0
    with open("mq_ntt_out.txt", "r") as NTT_out:
        for i in range(1024):
            golden = np.uint16(NTT_out.readline())
            if ntt_out[i] != golden:
                print(f"{i:<4} Failed!! Expect value: {golden}. Actual value: {ntt_out[i]}")
                error_cnt += 1
            #else:
                #print(f"Calculate value: {outBuffer0[i]:.17f}")
    if error_cnt == 0:
        print("Test Passed!!")
    else:
        print("Test Failed!!")
    
    print("===============================")
    print("Exit process")



Entry: /usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py"
0b10111111101000
Kernel execution time [mode: NTT]: 0.00023865699768066406 s
Kernel execution time [mode: FFT]: 0.00017070770263671875 s
-------------------------------
Compare FFT with golden vector:
Test Passed!!
-------------------------------
Compare NTT with golden vector:
Test Passed!!
Exit process
